# Day 1: Map with Points

> Map with point data (e.g., individual locations, points of interest, clusters). Focus on effective symbolization and density visualization.

Download the data:

```
overturemaps download -f geoparquet --type=water -o water.parquet
```

In [1]:
import sedona.db
import os
from lonboard import Map, ScatterplotLayer, viz
import wkls

In [11]:
os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

In [12]:
sd = sedona.db.connect()

In [22]:
# df = sd.read_parquet(
#     "/Users/matthewpowers/data/overture/water.parquet"
# )
df = sd.read_parquet(
    "s3://overturemaps-us-west-2/release/2025-09-24.0/theme=base/type=water/"
)

In [23]:
df.show(3)

┌──────────────────┬──────────────────┬─────────────────┬───┬──────────┬─────────┬─────────────────┐
│        id        ┆     geometry     ┆       bbox      ┆ … ┆ wikidata ┆ is_salt ┆ is_intermittent │
│       utf8       ┆     geometry     ┆      struct     ┆   ┆   utf8   ┆ boolean ┆     boolean     │
╞══════════════════╪══════════════════╪═════════════════╪═══╪══════════╪═════════╪═════════════════╡
│ d8b0ec9a-a24c-3… ┆ LINESTRING(-82.… ┆ {xmin: -82.027… ┆ … ┆          ┆         ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 470682d8-af2f-3… ┆ LINESTRING(-82.… ┆ {xmin: -82.025… ┆ … ┆          ┆         ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 04d7ee36-d786-3… ┆ LINESTRING(-82.… ┆ {xmin: -82.026… ┆ … ┆          ┆         ┆                 │
└──────────────────┴──────────────────┴─────────────────┴───┴──────────┴─────────┴─────────

In [24]:
df.to_view("base_water", True)

In [5]:
la = wkls.us.ca.losangeles.wkt()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [25]:
query = f"""
select ST_Centroid(geometry) as geometry from base_water
where
 ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{la}'), 4326))
 and source_tags['leisure'] = 'swimming_pool'
 and ST_GeometryType(geometry) = 'ST_Polygon'
"""

In [26]:
res = sd.sql(query)

In [ ]:
%%time
res.count()

In [31]:
layer = ScatterplotLayer(
    res,
    radius_min_pixels=1
)
m = Map(layer)

In [32]:
m

Map(custom_attribution='', layers=(ScatterplotLayer(radius_min_pixels=1.0, table=arro3.core.Table
+-----------…

In [33]:
viz(res)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

![map](../images/day01_mp_points_la_pools.png)

![map](../images/day01_mp_points_la_pools.png)

## LA Area

In [46]:
la_area = "POLYGON((-118.5672 34.143635, -117.719879 34.157273, -117.666321 33.685496, -118.56308 33.69121, -118.569946 34.107256, -118.5672 34.143635))"

In [47]:
query = f"""
select ST_Centroid(geometry) as geometry from base_water
where
 ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{la_area}'), 4326))
 and source_tags['leisure'] = 'swimming_pool'
 and ST_GeometryType(geometry) = 'ST_Polygon'
"""

In [48]:
res = sd.sql(query)

In [49]:
res.count()

22938

In [50]:
viz(res)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## Sao Paulo Area

In [51]:
sao_paulo = "POLYGON((-46.863556 -23.416628, -46.323853 -23.401505, -46.325226 -23.743869, -46.891022 -23.741355, -46.866302 -23.438049, -46.863556 -23.416628))"

In [52]:
query = f"""
select ST_Centroid(geometry) as geometry from base_water
where
 ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{sao_paulo}'), 4326))
 and source_tags['leisure'] = 'swimming_pool'
 and ST_GeometryType(geometry) = 'ST_Polygon'
"""

In [53]:
res = sd.sql(query)

In [54]:
res.count()

6055

In [55]:
viz(res)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## Phoenix pools

In [34]:
phoenix_wkt = "POLYGON((-112.8 33.0, -111.4 33.0, -111.4 33.9, -112.8 33.9, -112.8 33.0))"

In [35]:
query = f"""
select ST_Centroid(geometry) as geometry from base_water
where
 ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{phoenix_wkt}'), 4326))
 and source_tags['leisure'] = 'swimming_pool'
 and ST_GeometryType(geometry) = 'ST_Polygon'
"""

In [36]:
res = sd.sql(query)

In [37]:
res.count()

8001

In [38]:
viz(res)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## San Diego & Tijana pools

In [40]:
polygon = "POLYGON((-117.310638 32.760717, -116.838226 32.783811, -116.777802 32.448362, -117.159576 32.392718, -117.224121 32.617978, -117.310638 32.760717))"

In [41]:
query = f"""
select ST_Centroid(geometry) as geometry from base_water
where
 ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{polygon}'), 4326))
 and source_tags['leisure'] = 'swimming_pool'
 and ST_GeometryType(geometry) = 'ST_Polygon'
"""

In [42]:
res = sd.sql(query)

In [43]:
res.count()

1326

In [44]:
viz(res)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…